In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())['col_name']
data_tensor = torch.reshape(torch.tensor(data),(1,-1, 1))

create_dummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())['col_name']
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1,-1,1))

dummy_theory_output = torch.exp(torch.mean(torch.log(dummy_data_tensor)))
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))



In [6]:
scales = [8]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_38873/3257546855.py:8: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


In [8]:
# prover calculates settings, send to verifier
# In non-linearity, scale greatly affects lookup size
theory_output = torch.exp(torch.mean(torch.log(data_tensor)))
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        # self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Theory output:  tensor(47.6981)
==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[0,593654],"logrows":20,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":905,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,16],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":[{"Recip":{"scale":256.0}},{"Ln":{"scale":256.0}},{"Exp":{"scale":65536.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/t3/5psrvr1x0w1_6n9kx2n7d9700000gn/T/ipykernel_38873/4202400027.py:12: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.tensor(1), torch.exp(torch.mean(torch.log(X))))


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
# We can see here that it took huge resource, yet still inaccurate
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====

spawning module 0
spawning module 2


spawning module 0
spawning module 2


Time setup: 245.56888508796692 seconds
Theory output:  tensor(47.6981)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 92.68629455566406
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12773529019820117503, 7223165032241168505, 6388811546753741567, 1325363978837834469], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9340507686675262708, 12576478094795452846, 16595392862476743742, 1007129139500821923]]], 'proof': '179d52bf0251a56c68a8598e122fdf80ea20844d6f7b9d80fe8123a29344a2dd2a7748c09ee3f30cffd02a5d2fa0cee7f6647d76f201e9567d5a15c9687ff0d22e1cece22553ceac961011ebd185607caf8d377e6ca8b798e5639e892403d1582b66ff95afae2f4b9841222dc4938d5ff38f6eb695fd2cd43dbea59575678b812a273ab9fd719a9426ae91d2cdc7a0807bd502b0dcd7c9855b3cf0b88551dfa313fb2f441c1dc56de01614d73570536c6293588828a5747cfade63f783be71a626d8cecd2882b70ec738ed1d5bdc0927699cdbbf5915604c7cb4d209e22e0c532fda252164b43399d99605c1c56eba6c6f53aad05c1108a24f8fcc935ce29b6d0a2712937bc765ae4536a051fc8c1fe48042ee6436f1d96ce40d3640dc4e33ab1ccabb61d6840425ec59ccd790a627863a9b99947743c1a953ec9215599d841d1e5b0fcd012935c195dc76a2ed157faa35d5ab613fafd5a9102ecb99ec94eb

In [10]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

92.68629455566406